## Imports and reading the data

In [36]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [37]:
raw_df = pd.read_csv('../data/all_project_week_19_32.csv')
cols = ['project_slug', 'Category', 'Location', 'Goal_USD', 'Pledge_USD', 'Number_Backers', 'Staff_recommended' , 'Creator_nb_projects','Project_Community_top_countries','Project_description', 'Deadline', 'Launched_at' ]
raw_df = raw_df[cols]
raw_df = raw_df.dropna()

## Filtering projects that are still running when the data collection happened (2019 aug. 11)

In [38]:
# getting rid on unix timestamps
raw_df['end_date'] = pd.to_datetime(raw_df['Deadline'], unit='s', origin='unix')
raw_df['launch_date'] = pd.to_datetime(raw_df['Launched_at'], unit='s', origin='unix')

# Exploring how the launch- and end-dates look like in the dataset
print('earliest launch date: ', raw_df['launch_date'].min())
print('earliest deadline: ', raw_df['end_date'].min())
print('latest launch date: ', raw_df['launch_date'].max())
print('latest deadline: ', raw_df['end_date'].max())


earliest launch date:  2009-05-06 16:20:07
earliest deadline:  2009-06-24 22:26:00
latest launch date:  2019-07-30 10:03:18
latest deadline:  2019-09-28 08:11:41


In [39]:
# filtering on date of collection
cutoff_date = pd.Timestamp(year=2019, month=8, day=11, hour=0)
raw_df = raw_df[ raw_df['end_date'] < cutoff_date ]
raw_df = raw_df.drop(columns=['Deadline', 'Launched_at', 'launch_date', 'end_date'])
raw_df.columns

Index(['project_slug', 'Category', 'Location', 'Goal_USD', 'Pledge_USD',
       'Number_Backers', 'Staff_recommended', 'Creator_nb_projects',
       'Project_Community_top_countries', 'Project_description'],
      dtype='object')

## Creating target variable and adjusting other attributes

### Gettint the top countries by backers and location

In [40]:
# getting the top country and location to use as exo variable for STM
raw_df['top_country'] = raw_df.apply( lambda row: re.search(r'(?<=\|)(.*?)(?=\:)', row['Project_Community_top_countries']).group().lower() if re.search(r'(?<=\|)(.*?)(?=\:)', row['Project_Community_top_countries']) is not None else '' , axis=1)
raw_df['location'] = raw_df.apply( lambda row: re.search(r'(?<=\|).*', row['Location']).group().lower().strip() if re.search(r'(?<=\|).*', row['Location']) is not None else '' , axis=1)
raw_df.head()

,project_slug,Category,Location,Goal_USD,Pledge_USD,Number_Backers,Staff_recommended,Creator_nb_projects,Project_Community_top_countries,Project_description,top_country,location
0,the-because-black-life-conference-2018,Art,Minneapolis | US,10000.00000,10150.000000,72,True,1.0,|United States:57 backers|Australia:1 backer|S...,The Because Black Life Conference 2018 - This ...,united states,us
1,petography-illustrations-of-your-pet,Illustration,Victoria | CA,156.13997,645.084479,34,False,5.0,|United States:23 backers|Canada:2 backers|Uni...,Risks and challenges Having done print focuse...,united states,ca
2,dessert-cuties-enamel-pins,Art,Boston | US,800.00000,815.000000,33,False,2.0,|United States:24 backers|Australia:1 backer,Who doesn't love cute enamel pins? I madeÂ sev...,united states,us
3,adorable-deadpool-chibi-sticker,Illustration,Kettering | US,400.00000,870.000000,61,False,12.0,|United States:49 backers|United Kingdom:5 bac...,My name is Melanie (Melbaka) and welcome to my...,united states,us
4,japanese-mythos-enamel-pins,Art,Minneapolis | US,1050.00000,11069.000000,293,False,5.0,|United States:239 backers|Canada:11 backers|U...,â¦ï¸â¦ï¸â¦ï¸ Information â¦ï¸â¦ï¸â¦...,united states,us


### Calculating the percentage of amount actually pledged

In [41]:
raw_df['pledged_percentage'] = raw_df.apply(  lambda row: float(row['Pledge_USD'])/float(row['Goal_USD'])  , axis=1)
raw_df['pledged_binary'] = raw_df.apply(  lambda row:  1 if row['pledged_percentage'] >= 1 else 0, axis=1)
raw_df.head()

,project_slug,Category,Location,Goal_USD,Pledge_USD,Number_Backers,Staff_recommended,Creator_nb_projects,Project_Community_top_countries,Project_description,top_country,location,pledged_percentage,pledged_binary
0,the-because-black-life-conference-2018,Art,Minneapolis | US,10000.00000,10150.000000,72,True,1.0,|United States:57 backers|Australia:1 backer|S...,The Because Black Life Conference 2018 - This ...,united states,us,1.015000,1
1,petography-illustrations-of-your-pet,Illustration,Victoria | CA,156.13997,645.084479,34,False,5.0,|United States:23 backers|Canada:2 backers|Uni...,Risks and challenges Having done print focuse...,united states,ca,4.131450,1
2,dessert-cuties-enamel-pins,Art,Boston | US,800.00000,815.000000,33,False,2.0,|United States:24 backers|Australia:1 backer,Who doesn't love cute enamel pins? I madeÂ sev...,united states,us,1.018750,1
3,adorable-deadpool-chibi-sticker,Illustration,Kettering | US,400.00000,870.000000,61,False,12.0,|United States:49 backers|United Kingdom:5 bac...,My name is Melanie (Melbaka) and welcome to my...,united states,us,2.175000,1
4,japanese-mythos-enamel-pins,Art,Minneapolis | US,1050.00000,11069.000000,293,False,5.0,|United States:239 backers|Canada:11 backers|U...,â¦ï¸â¦ï¸â¦ï¸ Information â¦ï¸â¦ï¸â¦...,united states,us,10.541905,1


In [42]:
# getting to the final dataset
cols = ['project_slug', 'Category', 'location', 'Number_Backers', 'Creator_nb_projects', 'Project_description', 'top_country', 'pledged_percentage', 'pledged_binary']
raw_df = raw_df[cols]
raw_df.head()

,project_slug,Category,location,Number_Backers,Creator_nb_projects,Project_description,top_country,pledged_percentage,pledged_binary
0,the-because-black-life-conference-2018,Art,us,72,1.0,The Because Black Life Conference 2018 - This ...,united states,1.015000,1
1,petography-illustrations-of-your-pet,Illustration,ca,34,5.0,Risks and challenges Having done print focuse...,united states,4.131450,1
2,dessert-cuties-enamel-pins,Art,us,33,2.0,Who doesn't love cute enamel pins? I madeÂ sev...,united states,1.018750,1
3,adorable-deadpool-chibi-sticker,Illustration,us,61,12.0,My name is Melanie (Melbaka) and welcome to my...,united states,2.175000,1
4,japanese-mythos-enamel-pins,Art,us,293,5.0,â¦ï¸â¦ï¸â¦ï¸ Information â¦ï¸â¦ï¸â¦...,united states,10.541905,1


## Slicing dataset up on meta-categories

In [43]:
# Defining meta-categories according to kickstarter standards
art_meta = ['Art', 'Ceramics', 'Conceptual Art', 'Digital Art', 'Illustration', 'Installations', 'Mixed Media', 'Painting', 'Performance Art', 'Public Art', 'Sculpture', 'Social Practice', 'Textiles', 'Video Art']
comics_meta = ['Comics', 'Comic Books', 'Graphic Novels', 'Webcomics']
crafts_meta = ['Crafts', 'Candles', 'Crochet', 'DIY', 'Embroidery', 'Glass', 'Knitting', 'Pottery', 'Printing', 'Quilts', 'Stationery', 'Taxidermy', 'Weaving', 'Woodworking']
dance_meta = ['Dance', 'Performances', 'Residencies', 'Workshops']
design_meta = ['Design', 'Architecture', 'Civic Design', 'Graphic Design', 'Interactive Design', 'Product Design', 'Toys', 'Typography']
fashion_meta =['Fashion', 'Accessories', 'Apparel', 'Childrenswear', 'Couture', 'Footwear', 'Jewelry', 'Pet Fashion', 'Ready-to-wear']
film_meta = ['Film', 'Film &amp; Video', 'Action', 'Animation', 'Comedy', 'Documentary', 'Drama', 'Experimental', 'Family', 'Fantasy', 'Festivals', 'Horror', 'Movie Theaters', 'Music Videos', 'Narrative Film', 'Romance', 'Science Fiction', 'Shorts', 'Television', 'Thrillers', 'Webseries']
food_meta = ['Food', 'Bacon', 'Community Gardens', 'Cookbooks', 'Drinks', 'Farms', 'Food Trucks', 'Food Trucks', 'Restaurants', 'Small Batch', 'Spaces', 'Vegan']
games_meta = ['Games', 'Gaming Hardware', 'Live Games', 'Mobile Games', 'Playing Cards', 'Puzzles', 'Tabletop Games', 'Video Games']
journalism_meta = ['Journalism', 'Audio', 'Photo', 'Print', 'Video', 'Web']
music_meta = ['Music', 'Blues', 'Chiptune', 'Classical Music', 'Comedy', 'Country &amp; Folk', 'Electronic Music', 'Faith', 'Hip-Hop', 'Indie Rock', 'Jazz', 'Kids', 'Latin', 'Metal', 'Pop', 'Punk', 'R&amp;B', 'Rock', 'World Music']
photography_meta = ['Photography', 'Animals', 'Fine Art', 'Nature', 'People', 'Photobooks', 'Places']
publishing_meta = ['Publishing', 'Academic', 'Art Books', 'Calendars', 'Children&#39;s Books', 'Comedy', 'Fiction', 'Letterpress', 'Literary Journals', 'Literary Spaces', 'Nonfiction', 'Periodicals', 'Poetry', 'Radio &amp; Podcasts', 'Translations', 'Young Adult', 'Zines']
technology_meta = ['Technology', '3D Printing', 'Apps', 'Camera Equipment', 'DIY Electronics', 'Fabrication Tools', 'Flight', 'Gadgets', 'Hardware', 'Makerspaces', 'Robots', 'Software', 'Sound', 'Space Exploration', 'Wearables', 'Web']
theater_meta = ['Theater','Comedy', 'Experimental', 'Festivals', 'Immersive', 'Musical', 'Plays']

meta_categories = [art_meta, comics_meta, crafts_meta, dance_meta, design_meta, fashion_meta, film_meta, food_meta, games_meta, journalism_meta, music_meta, photography_meta, publishing_meta, technology_meta, theater_meta]
cat_df_dict = {}
for category in meta_categories:
  df_1 = raw_df[raw_df['Category'].isin(category)]
  cat_df_dict.update( {str(category[0]): df_1} )

In [44]:
# exploring category numbers
for key in cat_df_dict:
  print(key + ' category has ' + str(len(cat_df_dict[key])) + ' rows')

Art category has 8916 rows
Comics category has 1925 rows
Crafts category has 1728 rows
Dance category has 1682 rows
Design category has 3588 rows
Fashion category has 2112 rows
Film category has 8824 rows
Food category has 1761 rows
Games category has 5165 rows
Journalism category has 1011 rows
Music category has 9651 rows
Photography category has 1777 rows
Publishing category has 6677 rows
Technology category has 4089 rows
Theater category has 1707 rows


## Subsampling dataset
According to [statista.com](https://www.statista.com/statistics/222455/amount-of-dollars-pledged-per-category-on-kickstarter/), the three most funded categories were *Games*, *Design*, and *Technology*, so we will focus our efforts on these. We will test descriptive stats to figure out the best subsampling strategy.

In [91]:
def generate_descriptive_statistics(dataframe):
    name =[x for x in globals() if globals()[x] is dataframe][0]
    mean_of_cont = dataframe['pledged_percentage'].mean()
    var_of_cont = dataframe.var()['pledged_percentage']
    std_of_cont = dataframe.std()['pledged_percentage']
    binary_ratio = float(dataframe['pledged_binary'].value_counts()[0])/float(dataframe['pledged_binary'].value_counts()[1])
    print('----------------------')
    print('in the ', name, 'dataframe the  ratio of the binary target counts:')
    print(binary_ratio)
    print('in the ', name, 'dataframe the mean of cont. target var: ', mean_of_cont, ', std: ', std_of_cont)


In [110]:
games_df = cat_df_dict['Games']
games_sub = games_df.sample(n=3500)

In [111]:
generate_descriptive_statistics(games_df)
generate_descriptive_statistics(games_sub)

----------------------
in the  games_df dataframe the  ratio of the binary target counts:
0.10127931769722814
in the  games_df dataframe the mean of cont. target var:  21.8255945773875 , std:  638.9184219503052
----------------------
in the  games_sub dataframe the  ratio of the binary target counts:
0.10270951480781348
in the  games_sub dataframe the mean of cont. target var:  19.309524954583196 , std:  704.8549737286678


C:\Users\bocsa\AppData\Local\Temp/ipykernel_32820/4179898495.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  var_of_cont = dataframe.var()['pledged_percentage']
C:\Users\bocsa\AppData\Local\Temp/ipykernel_32820/4179898495.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  std_of_cont = dataframe.std()['pledged_percentage']


In [ ]:
design_df = cat_df_dict['Design']
design_sub = design_df.sample(n=3500)

In [95]:
generate_descriptive_statistics(design_df)
generate_descriptive_statistics(design_sub)

----------------------
in the  design_df dataframe the  ratio of the binary target counts:
0.20281595709017766
in the  design_df dataframe the mean of cont. target var:  6.24611912653669 , std:  24.3941190415017
----------------------
in the  design_sub dataframe the  ratio of the binary target counts:
0.2006861063464837
in the  design_sub dataframe the mean of cont. target var:  6.263030799771488 , std:  24.60503939955467


C:\Users\bocsa\AppData\Local\Temp/ipykernel_32820/4179898495.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  var_of_cont = dataframe.var()['pledged_percentage']
C:\Users\bocsa\AppData\Local\Temp/ipykernel_32820/4179898495.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  std_of_cont = dataframe.std()['pledged_percentage']


In [96]:
tech_df = cat_df_dict['Technology']
tech_sub = tech_df.sample(n=3500)

In [97]:
generate_descriptive_statistics(tech_df)
generate_descriptive_statistics(tech_sub)

----------------------
in the  tech_df dataframe the  ratio of the binary target counts:
0.3076431084106172
in the  tech_df dataframe the mean of cont. target var:  4.229911844795767 , std:  26.097965303366713
----------------------
in the  tech_sub dataframe the  ratio of the binary target counts:
0.3049962714392245
in the  tech_sub dataframe the mean of cont. target var:  4.2240070472287155 , std:  27.812399716918126


C:\Users\bocsa\AppData\Local\Temp/ipykernel_32820/4179898495.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  var_of_cont = dataframe.var()['pledged_percentage']
C:\Users\bocsa\AppData\Local\Temp/ipykernel_32820/4179898495.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  std_of_cont = dataframe.std()['pledged_percentage']


In [112]:
games_sub.to_csv('../data/raw_games_data.csv')
design_sub.to_csv('../data/raw_design_data.csv')
tech_sub.to_csv('../data/raw_tech_data.csv')

## Text preprocessing

The preprocessing will be done using the following function but we run it on a vm to make it faster.

In [14]:
def preprocess(text):
    if isinstance(text, str):
        lemmatizer = WordNetLemmatizer()
        text = text.strip()
        result = ""
        punctless = re.sub('[^a-zA-Z0-9]', ' ', text)
        lowercase = punctless.lower()
        words_list = lowercase.split()
        words_list =  [ lemmatizer.lemmatize(word) for word in words_list if not word in set(stopwords.words('english')) ]
        for word in words_list:
            result += " " + word.strip()
        return result
    else: return ''

## TO DO: joining CLEAN datasets with pledge table data, and add in 'Staff_Recommended' variable